# 5. Geth 거래

Last Updated 20190508WEB1530 20180717TUE1700 20160207

## 학습내용

### 목표

- 계정, 트랜잭션, 암호화, 마이닝, 합의 이해

### 목차

* 5.1 [메시지](#5.1)
* 5.1.1 call
* 5.1.2 sendTransaction
* 5.4 ABI 명세
* 5.3 트랜잭션 구성
* 5.3.1 비트코인의 트랜잭션
* 5.3.2 트랜잭션
* 5.4 트랜잭션 처리 단계
* 5.5 블록체인
* 5.6 마이닝
* 5.7 gas


### 문제

* 문제 5-1: [다른 계정으로 송금 거래](#5-1)


## 5.1 메시지
<a id='5.1'></a>

이더리움에서의 메시지는 **계정 간**, 예를 들면 **자신의 계정에서 다른 사람의 계정 또는 컨트랙 계정**으로 
전달되는 **데이터**와 **value**를 말한다.

메시지, Message는 우리가 자주 사용하는 핸드폰의 **문자 메시지**,
내용을 작성한 후 상대 번호를 지정하고 전송하는 서비스와 비슷하다.
핸드폰의 문자메시지와 이더리움 메시지를 서로 비교해 보자.

구성 | 핸드폰 | 이더리움 메시지 | 비교
-----|-----|-----|-----
데이터 | 텍스트 | **바이트코드** | 핸드폰 문자는 사람이 읽을 수 있는 텍스트이고 이더리움 메시지는 EVM에서 처리할 수 있는 바이트코드로 작성한 메시지 또는 컨트랙 호출.
value | 송금 | 입금, 송금 등 금액 | 휴대폰 번호 송금, 반면 이더리움에서는 입금 또는 송금
송신 | 전화번호 또는 컴퓨터 (SNS, Email) | 내, 외부계정 | 고유번호 또는 사용자계정
수신 | 전화번호 또는 컴퓨터 (SNS, Email) | 내, 외부계정 | 상동


메시지는 계정의 사인이 필요한지에 따라 1) 메시지 call과 2) 트랜잭션으로 구분한다.
**메시지 call은 사인을 하지 않고**, **트랜잭션은 사인**을 해야 한다.
다시 말하면, 트랜잭션은 전송자에 의해 사인이 된 메시지를 말한다.

함수 | 설명 | 블록체인에 기록
-----|-----|-----
call | 로컬에서 실행, 블록체인에 전송되지 않는다. 따라서 블록체을 변경하지 않고, 읽기만 하는 경우, 비용이 발생하지 않음 | NO
sendTransaction | 외부계정에 의해 발생하고, 사인해서 블록체인에 전송된다. 블록체인을 변경할 경우 사용 | YES 반환값은 hash.

또한 메시지를 보내는 측, **외부계정에서 발생하는지 또는 내부계정에서 발생하는지에 따라 메시지를 구분**할 수 있다.
외부계정, 예를 들어 갑돌이 계정에서 **contract**으로 보낼 수 있다. 이 경우 받는 주소가 블록체인 상의 contract이므로 해당 소스코드가 실행된다. 또는 **상대방 계정**, 예를 들어 갑순이 계정으로 직접 전송할 수도 있다.


### 5.1.1 call
call() 함수는 로컬 블록체인에서의 호출이라 블록체인에 전송되거나 기록되지 않는다.
따라서 **사인이 필요없고**, gas가 발생하지 않는다.
따라서 네트워크 참여자가 그 함수가 호출되었는지 알지 못한다.
call() 함수는 실행되고 나서 그 반환 값을 가질 수 있다.
web3.js API는 **web3.eth.call**이다.

내부 계정, 즉 contract은 다른 contract이나 계정으로 메시지 call을 할 수 있다.
그 예는:

```python
call.gas(0).value(number)()
```

> Solidity Documentation의 메시지 호출 설명

> 컨트랙은 다른 컨트랙을 호출하거나 다른 계정으로 메시지 호출이라는 수단을 통해 Ether를 전송할 수 있다.
메시지 호출은 트랜잭션과 비슷한데 송신, 수신, 데이터, Ether, gas, 반환 값을 가진다.
Contracts can call other contracts or send Ether to non-contract accounts by the means of **message calls**. Message calls are similar to transactions, in that they have a source, a target, data payload, Ether, gas and return data. 

### 5.1.2 sendTransaction

sendTransaction() 함수는 블록체인에 전송되고 기록된다.
**사인**을 해야 하고, gas가 필요하다.
트랜잭션은 반환 값이 없고, 그 결과 값은 해시 값이다. 값을 알려면 event를 사용한다.
해시 값이 주어지면, 트랙잭션은 마이닝이 되어야 하고 반환 값은 없다.
web3.js API는 **web3.eth.sendTransaction**이다.

```python
coin.send(web3.eth.accounts[0], 100,{from:web3.eth.accounts[0],gas:100000});
coin.send.sendTransaction(web3.eth.accounts[0], 100,{from:web3.eth.accounts[0],gas:100000});
```

> Solidity Documentation의 트랜잭션 설명

> 트랜잭션이 메시지인지 구분이 모호할 수 있다. 공식 문서를 참고하면 도움이 되겠다.
> 트랜잭션은 한 계정에서 다른 계정으로 전송되는 메시지이다. 바이너리 데이터와 Ether를 포함할 수 있다.
원문을 읽어보면 "A transaction is a message that is sent from one account to another account. It can include binary data (which is called "payload") and Ether"


## 5.2 ABI 명세

스마트계약의 함수호출은 함수 및 인자를 부호화해서 호출한다.
외부에서 호출하거나 내부에서 컨트랙간에 호출하는 경우에 모두 해당이 된다.

예를 들어, Foo의 baz()함수를 호출한다고 하자.
우선 함수와 인자를 각 각 인코딩하고 조합해서 16진수 데이터로 만들어 호출하게 된다.
출처: https://solidity.readthedocs.io/en/develop/abi-spec.html
```python
contract Foo {
  function bar(fixed[2] xy) {}
  function baz(uint32 x, bool y) returns (bool r) { r = x > 32 || y; }
  function sam(bytes name, bool z, uint[] data) {}
}
```

1) 함수명 부호화
호출할 함수명(인자포함)을 해시값으로 만들고, 앞 4바이트를 선택한다. 이를 **function selector**라고 한다.
함수 시그니처 ```baz(uint32,bool)```의 Keccak hash ASCII 앞 4바이트를 선택한다.
```python
> web3.sha3('baz(uint32,bool)').substring(0,10);
"0xcdcd77c0"
```

2) 인자 부호화
값을 부호화한다. 32바이트로 만들기 위해 앞에 0으로 채운다.
인자가 69일 경우, 앞은 0으로 채워서 32바이트로 만든다.
```python
> web3.toHex(69)
"0x45"
0x0000000000000000000000000000000000000000000000000000000000000045 - 인자 uint32 69 (32바이트)
0x0000000000000000000000000000000000000000000000000000000000000001 - 인자 bool 1  (32바이트)
```

3) 데이터 조합
위 1) function selector부호값과 2) 인자부호값를 합친다. 0x를 제거한다.
```python
0xcdcd77c00000000000000000000000000000000000000000000000000000000000000045
0000000000000000000000000000000000000000000000000000000000000001
```

4) 함수 호출
```python
web3.eth.sendTransaction({
    from: eth.accounts[0],
    to: "0x672807a8c0f72a52d759942e86cfe33264e73934",
    data: "0xcdcd77c000000000000000000000000000000000000000000000000000000000000000450000000000000000000000000000000000000000000000000000000000000001",
    gas: 400000}
)
```

## 5.3 트랜잭션 구성

### 5.3.1 비트코인의 트랜잭션

비트코인의 거래는 버전, locktime 타임스탬프, 입금, 출금으로 구성한다.
입금, 출금이 하나가 아니고 여러 개인 점에 주의한다.
잔액이 흩어져 있어서 입금이 여럿이다.
출금 또한 수신자가 1명이라도, 그 잔액이 자신에게 돌려지므로 최소 2개가 된다.

길이 | 항목 | 설명
-----|-----|-----
4 bytes | Version | 버전정보
1-9 bytes | Input counter | in 개수
variable | in | array로서 이전 거래의 out에서 가져와 본 거래의 out으로 지급된다.사용하게 되는 잔액을 가지고 있는 transactionHash, script를 가진다.
1-9 bytes | output counter | out 개수
variable | out | array로서 in보다 같거나, 적어야 한다 (적으면 처리비용으로). UTXO OpCodes: Scripts - 6...4사용하고 남는 잔액들 unspent transaction output UTXO. script, value를 가진다.
4 bytes | locktime | unix timestamp 또는 블록번호

In [2]:
import bitcoin, pprint
tx='0100000002349cbebe5628707553812eb0e591dc49047d743e836006773a65b55cf4444a5a030000006a47304402203f02d209168bb9c4e8fab3b2f2aca19be1f9032eaa85be3a029641eecf69e60d022055d7e22914bb4741ee8c1ab73e23766de034a82dae22dad0c13684d6cb098b9e012102c3a7056ad278bf04f2e5546e29466d8175ec4a052ad8f9901b32d53e442545a8ffffffff0890e18ae1953971b7b64ddedcb75c034e5c16969e498358b517960fd0186238010000006b4830450221009511497e136765037a75f75a126168502bfee73a7c286ea775968a7140368ef002200cfc776ade17b9444bb7ad13e71852974befd1d6fcfa196df677cd72a697c63b012102b8963213667c71471d62361fe27aad7ab2b8b102a6d93f7367634466e04b5422ffffffff022085d0020000000017a914ed91639c578b4c1f526525e6dcbc9fbc505fe7698751bd0d00000000001976a9145f9a3e6e8029dd934a2a00a17aa8254c0975219988ac00000000'
tx_structure = bitcoin.deserialize(tx)
pprint.pprint(tx_structure)

{'ins': [{'outpoint': {'hash': '5a4a44f45cb5653a770660833e747d0449dc91e5b02e815375702856bebe9c34',
                       'index': 3},
          'script': '47304402203f02d209168bb9c4e8fab3b2f2aca19be1f9032eaa85be3a029641eecf69e60d022055d7e22914bb4741ee8c1ab73e23766de034a82dae22dad0c13684d6cb098b9e012102c3a7056ad278bf04f2e5546e29466d8175ec4a052ad8f9901b32d53e442545a8',
          'sequence': 4294967295},
         {'outpoint': {'hash': '386218d00f9617b55883499e96165c4e035cb7dcde4db6b7713995e18ae19008',
                       'index': 1},
          'script': '4830450221009511497e136765037a75f75a126168502bfee73a7c286ea775968a7140368ef002200cfc776ade17b9444bb7ad13e71852974befd1d6fcfa196df677cd72a697c63b012102b8963213667c71471d62361fe27aad7ab2b8b102a6d93f7367634466e04b5422',
          'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': 'a914ed91639c578b4c1f526525e6dcbc9fbc505fe76987',
           'value': 47220000},
          {'script': '76a9145f9a3e6e8029dd934a2a00a17aa8254c0975219

### 5.3.2 트랜잭션

#### 거래 속성

거래를 발생하려면 필요한 속성이 채워져야 한다.

속성 | 바이트 | 설명
----|-----|-----
from | 20 바이트 | 전송 주소. 명시하지 않으면 web3.eth.defaultAccount
to | 20 바이트 | (optional) 수신 주소. 컨트랙 생성하는 경우는 당연히 명시하지 않아도 된다.
value | 32 바이트 | (optional) 전송 Wei 금액
gasLimit | 32 바이트 | (optional, default: To-Be-Determined) 거래처리에 허용되는 최대 개스량
gasPrice | 32 바이트 | (optional, default: To-Be-Determined) 거래 처리에 필요한 개스량 (Wei)
data | 제한 없슴 | (optional)  메시지 데이터 (바이트)
nonce | 32 바이트 | (optional) 계정에서 발생한 거래 순서 번호. 동일한 번호가 있으면 하나는 처리 거절, 또는 순서대로 처리 (즉 뒷 순서가 먼저 처리되고 앞 번호를 취소하는 이중거래 편법을 막음)
V | 1 | ECDSA 사인 관련
R | 32 바이트 | (r, s) is the normal output of an ECDSA signature, where r is computed as the X coordinate of a point R, modulo the curve order n.
S | 32 바이트 | 

예를 들면:

```python
transactionObject = {
  from: "0xA0A0A0A01",
  to: "0xA0A0A0A02",
  value: web3.eth.getBalance("0xA0A0A0A01"),
  gas: "----",
  gasPrice: "----",
  data: "----",
  nonce: 0,
}
```

#### Opcodes

컴퓨터로 하여금 실행할 수 있는 기계어와 같은 수준의 명령코드 **Op**eration **Codes**를 말한다.
비트코인에서는 stack based 스택기반의 언어, 머신코드 명령어 Opcodes를 스택에 push, pull로 실행한다.
* ```2 3 OP_ADD 5 OP_EQUAL``` 2+3==5
* ```2 3 OP_ADD 2 OP_MUL 1 OP_ADD 11 OP_EQUAL``` ((2+3)*2)+1==1
* 기본적인 2개의 스크립트
    * scriptPubKey: locking script, pubKey로 잠그는 명령문
    ```DUP HASH160 <PubHash> EQAULVERIFY CHECKSIG```
    * scriptSig: unlocking script, pubKey를 푸는 sig로 해제하는 명령문. 
    <sig> <PubK>
* P2PKH Pay-to-Public-Key-Hash script
    * scriptSig, scriptPubKey를 합쳐서 실행
* Pay-to-Public-Key
     <Signature from Private Key A> <Public Key A> OP_CHECKSIG

## 5.4 트랜잭션의 처리 단계

갑이 을에게 송금 거래를 한다고 하자.
갑은 거래에 디지털서명을 하고
그 거래는 참여 노드들에게 전파된다.
거래의 디지털서명을 인증하여 원본이 맞는지, 전송자가 맞는지 확인한다.
거래를 블록으로 만들어 가고, 만들면 모든 네트워크참여자에게 공지한다.
블록이 체인에 연결, 노드들이 블록해시를 다음 블록에 참조하면서 연결이되고 이를 받아들인 것으로 합의한다.

순서 | 단계 | 설명
-----|-----|-----
1 | 거래 생성 | 거래에 **디지털 서명을**하고, 전송하기 위해 바이트 단위로 변환, **직렬화 serialize**한다.
2 | 거래 전송 | p2p 네트워크로 전송, 노드들 간에 서로 전파 (broadcast)
3 | 블럭 생성 | 인증된 거래가 블럭으로 만들어짐. 아직 인증되기 전이므로 candidate block
3-1 | 인증 | sender의 public key로 소유주 서명 맞는지? (전송과정 위조가 없다), 블록 nonce 인증 well-formed
3-2 | 대기 | pendingTransaction에 넣음. 모아서 블록이 될 때까지. 로컬메모리에 transaction pool, orphan pool
3-3 | 블록 후보 | 로컬에서 만들어진 블록 후보를 네트워크로 공지. candidate block를 pending block<p>마이너는 자신들이 거래를 수집하여 블록으로 만들기 때문에 서로 같지 않을 수 있다.<p>거래가 도착 순서대로 묶어지는 것이 아니라, 부모hash가 없는 orphan transactions이 있을 수 있다.<p>우선순위가 높은 (잔고UTXO의 age, 금액이 클수록 높음) 거래부터 시작해서 블록으로 만들어짐. Priority = Sum (Value of input * Input Age) / Transaction Size.<p>포함이 안된 거래는 기다려서 다음 블록에 추가.
4 | 블록인증 | 마이닝을 해서 블록을 인증
4-1 | 마이닝 | Proof of Work hash puzzle을 풀어 nonce를 정함.
4-2 | 블록인증 |  블록을 인증. 블록이 인증되면 다른 마이너 종료. 블록을 앞 블록에 전의 블록 hash를 찾음 체이닝. 2016개마다 문제의 난이도를 조정한다.
4-3 | 보상 | 2종류의 보상: 1) 새롭게 생성된 코인을 획득 (2015년 25비트코인 생성), 2) 거래비용
5 | 새로운 블록 전파 | 새로운 블럭이 공지되면 다른 마이닝은 종료. 거래기록은 중앙 서버가 가지고 있는 것이 아니라, 분산 네트워크에 실시간 공유된다. 공유되고 나면 수정할 수 없다. 로컬 블럭체인 동기화, 마이닝하지 않은 경우 동기화가 뒤떨어질 수 있다.

## 5.5 블록체인

### 블록헤더

블록체인은 블록의 연결로 헤더와 바디로 구분된다. 블록헤더는:
1) 프로토콜 버전(Version), 2) 이전 블록 해시 값 (Previous Block Hash), 3) 채굴속도를 조정하는 난이도 (Difficulty), 4) 블록이 생성된 시간 (Timestamp), 5) Target Hash를 맞추기 위해 사용하는 난스(Nonce), 6) 각 블록의 해시 값을 결합하여 계산하는 머클 루트의 해시값 (Merkle Root)으로 구성한다. 블록바디는 발생한 트랙잭션을 저장한다.

블록헤더 | 바이트 수 | 설명
-----|-----|-----
version | 4 | 프로토콜 버전
Previous Block Hash | 32 | 이전 블록 해시 값으로 서로 체인으로 연결.
 Merkle Root | 32 | 블록에 있는 모든 거래 hash의 root. 따라서 어떤 블럭 해시 값이 변경되면 Merkle Root Hash값도 달라지게 되어 수정이 불가능함.
Timestamp | 4 | 블록 생성 시간
difficulty target | 4 | 난이도를 조정하여 블록생성시간을 유지.
nonce | 4 | Target Hash를 맞추어 나갈 때 조정/

### 블록체인

블록체인은 블록의 체인이고, 블록은 **거래를 묶어 이진트리 merkle tree로 구성**해서 가진다.
예를 들어, 트랜잭션 A, B, C를 머클트리로 구성한다면 각 거래 데이터를 해싱해서 나온 결과 값이 각 노드에 저장된다. 부모 노드를 만들기 위해서 자식노드의 32바이트 크기의 트랜잭션 A와 B의 해시 값을 서로 연결해서 64바이트 문자열을 만들고 문자열은 이중 해시 처리되어 부모노드의 해시를 생성한다. 
이러한 방식으로 상위에 노드가 하나 남을 때까지 계속 계산하고, 각각의 트랜잭션을 이진트리 형태로 만들 경우 가장 최종적으로 남는 해시 값이 머클트리 루트 해시 값이 된다.

블록 내에서 머클 해시 값을 통해서 트랜잭션의 무결성을 검증 할 수 있고, 블록 해시 값을 통해서 해더 값에 대한 무결성을 검증할 수 있다.
거래 내역의 위변조를 막기 위해서 해시로 만들고 이것을 트리 형태로 만든 것이다.
어느 한 블록에 변조가 발생하면, merkle tree로 root까지 전달되어 블록체인 전체가 변조된다.
이전에 생성된 블록해시 값은 다음 블록 헤더의 'Previous Block Hash' 항목에 저장된다. 각 블록에 트랜잭션이 지속적으로 쌓이면 새롭게 블록을 생성하여 블록해시 값을 연결고리로 블록을 연결하여 블록체인이 생성된다.
블록 뒤에 다른 블록이 추가될 때마다, 이를 **confirmation**이라고 하는데 confirmation이 늘어날 수록 수정이 힘들어 진다. 
**fork**는 **같은 블록높이를 가지는 블록이 여러 개**, 부모가 여러 자식 블록을 가지는 경우 발생한다. 복수의 마이너가 거의 동시에 블록을 생산하는 경우이다.
매 **2016블록마다 difficulty가 조정**된다.


> 블록높이 block height

> 블록체인에서 선행블록의 수를 말한다. 예를 들어 genesis block의 블록높이는 0이다.

source: bitcoinwiki https://en.bitcoin.it/wiki/Block_hashing_algorithm

In [1]:
import hashlib
header_hex = ("01000000" +
 "81cd02ab7e569e8bcd9317e2fe99f2de44d49ab2b8851ba4a308000000000000" +
 "e320b6c2fffc8d750423db8b1eb942ae710e951ed797f7affc8892b0f1fc122b" +
 "c7f5d74d" +
 "f2b9441a" +
 "42a14695")
header_bin = header_hex.decode('hex')  # changes to binary string
hash = hashlib.sha256(hashlib.sha256(header_bin).digest()).digest()

In [3]:
hash.encode('hex_codec')    # equals to "encode('hex')"

'1dbd981fe6985776b644b173a4d0385ddc1aa2a829688d1e0000000000000000'

In [4]:
hash[::-1].encode('hex_codec')  # [::-1] all elements in the array reversed

'00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'

### Merkle proof

머클트리 해시를 만드는 과정. 
특정 transaction이 수정되면 전체 해시가 다르게 된다.

```python
                              Merkle Root
                    /
                   /
                Hash 1 2
          /double SHA2     \
         /hash1+2 byte swap \
    byte swap            byte swap
      Hash 1               Hash 2
        |                    |
        | double SHA256      | double SHA256  
Transaction 1 Data   Transaction 2 Data
```

hash를 구하고, byte swap을 해주어야 한다.

In [18]:
t1 = "8c14f0db3df150123e6f3dbbf30f8b955a8249b62ac1d1ff16284aefa3d06d87"
t1swap="".join(reversed([t1[i:i+2] for i in range(0, len(t1), 2)]))
print t1swap

876dd0a3ef4a2816ffd1c12ab649825a958b0ff3bb3d6f3e1250f13ddbf0148c


In [19]:
t2 = "fff2525b8931402dd09222c50775608f75787bd2b87e56995a7bdd30f79702c4"
t2swap="".join(reversed([t2[i:i+2] for i in range(0, len(t2), 2)]))
print t2swap

c40297f730dd7b5a99567eb8d27b78758f607507c52292d02d4031895b52f2ff


In [22]:
t12=t1swap+t2swap
print t12

876dd0a3ef4a2816ffd1c12ab649825a958b0ff3bb3d6f3e1250f13ddbf0148cc40297f730dd7b5a99567eb8d27b78758f607507c52292d02d4031895b52f2ff


In [31]:
import hashlib
t12_bin=t12.decode('hex')
doubleHash12=hashlib.sha256(hashlib.sha256(t12_bin).digest()).digest()
doubleHash12.encode('hex')   # or doubleHash12.encode('hex_codec')

'15b88c5107195bf09eb9da89b83d95b3d070079a3c5c5d3d17d0dcd873fbdacc'

## 5.6 마이닝

마이닝은 거래를 인증하는 과정으로, hash puzzle을 풀고 그 보상으로 새로운 코인을 생성하게 된다.
발생한 모든 거래가 추가되는 것은 아니다.
**목표 값 target hash를 찾아내는 계산**을 하고, 맞추는 경우에 인증되고 블록체인에 비로서 추가된다.
정답을 맞추는 작업을 하는 참여자들을 마이너라고 하며, **보상은 정답을 맞춘 최초의 경우**에만 해당 마이너에게 주어진다.
이와 같이 일정한 량의 계산에 따라 거래가 인증되고, 블록체인에 추가될 수 있으므로 이러한 **계산작업증명을 Proof of Work** 이라고 한다.

비트코인의 경우 블록생성시간은 10분으로 설정되어 있으므로 하루 144 블록, 일주 1008블록이 생성된다. 
이더리움은 문제의 난이도는 12초에 1건씩 풀도록 정한다.

> Orhan, Stale

> * **Orphans**  블록체인에서 부모가 없는 블록을 말한다. 부모로 삼으려 했던 블록이 인증이 실패하면서 부모가 없어지는 경우 발생한다.
* **stale** 블록은 자식이 없는 블록을 말한다. 마이닝은 되었지만, 블록높이가 동일한 다른 블록이 먼저 자식이 생기면서 정상적인 블록체인이 되는 경우 발생한다.

이더리움에서의 합의 알고리즘은 GHOST (Greedy Heaviest Observed Subtree, Zohar and Sompolinsky in December 2013)이다. 가장 무거운 트리가 블록체인에 연결되는 방식이다. 비트코인은 가장 긴 블록을, 반면에 이더리움은 가장 무거운 블록을 추가한다.

> Hash Rate

> Hash Rate는 1초에 Hash를 몇 회하는지 (Number of hashes per second, H/s)
마이닝하는 속도를 말한다. 성능이 좋은 컴퓨터는 당연히 Hash Rate이 높게 마련이다.
H/s는 일반 척도와 같은 단위를 사용하여 나타낼 수 있다.

구분 | 단위 | 해시 회수
-----|-----|-----
Kilohashes | KH/s | 1000
Megahashes | MH/s | 1,000,000
Gigahashes | GH/s | 1,000,000,000
Terahashes | TH/s | 1,000,000,000,000
Petahashes | PH/s | 1,000,000,000,000


**Ethash**
이더리움이 구현한 PoW 알고리즘이다.
이전에 사용하였던 Dagger-Hashimoto 알고리즘을 개선한 버전이다.
Ethash의 해시함수를 "sha3_256" and "sha3_512"이라고 한다.

Ethash는 Ethash DAG라고 하는 1 GB 데이터 분량을 사용한다. 전체를 가지고 있을 필요가 없는 light client는 16 MB 캐시를 사용한다.
이 데이터는 매 **30,000 블록마다 재생성**되는데, 이 시간을 **epoch**이라고 한다.
마이닝할 때 DAG의 일부를 가져다가 블럭해시와 nonce를 사용해 target hash를 맞춰 나갈 때 사용한다.

마이닝을 하기 전에 보상이 지급되는 계정 coinbase를 정해야 한다.
coinbase를 충전할 계정으로 변경하고, mining. 그렇지 않으면 reward가 주어질 계정이 없다.
처음 채굴을 시작하면, 기가규모의 블럭체인을 내려받아야 한다. 약 10분 정도 소요되며, DAG가 구성된다.

```python
> miner.setEtherbase(eth.accounts[2])
true
> eth.coinbase
"0x53cbba17cf9bd0735855809bdcb88e232de96f32"
> miner.start(1);admin.sleepBlocks(1);miner.stop()
```

**hash puzzle**은 **block hash**값을 결정하기 위한 문제이다.
즉, **```sha256(sha256(data+nonce)) < difficulty```**이 되도록 nonce를 찾아야 한다.M
그 절차를 좀 더 자세하게 설명하면:
1. 거래를 수집하여 블록을 만든다 (비트코인은 1MB 블록)
2. 블록헤더 (Version + Previous Block Hash + Merkle Root + Timestamp + Difficulty Bits + Nonce) 값을 SHA-256 해시하고 또 재해시를 하여 해시값을 계산한다.
3. 2에서 계산된 해시값을 Target hash와 비교한다. target hash보다 적으면 정답, 아니면 1번으로 돌아감.
4. 문제를 푼 경우, 
문제를 풀면 블럭체인에 승인된 블럭을 맨 뒤에 첨부하고 모든 peer에게 공지.
참여자들에게 공지하고 참여자들로 하여금 계산을 하게 하여 검증되면 합의한다.
5. 참여자가 많아져 계산속도가 빨라질 수 있지만 설정된 블럭생성시간에 맞추어 난이도가 조정된다.
즉, 부모블록과 현재블록의 생성시간이:
    * 10초 이하 -> 난이도 증가
    * 10~19초: 유지
    * 20초 이상 -> 난이도 감소

6. 블록이 한꺼번에 2개 마이닝 되는 경우, 
두 마이너가 동시에 풀면 fork가 발생
난이도가 높은 블록이 인증받게 된다.
```python
loop
    if sha256(sha256(data+nonce)) < difficulty
        stop
    else change the nonce
```

블록헤더 데이터의 해시 값에 NONCE를 더하여 
앞 자리의 0의 개수를 맞출 때까지 반복한다.

In [ ]:
import hashlib
ntry=1
found=False
blockNumber=54
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z).hexdigest()
    if guessHash[:4]=='00000':
        found=True
    NONCE+=1
print guessHash
print NONCE

* 마이닝이 안되는 경우
    * 발생한 트랜잭션이 처리되지 않고 여러 건이 적체되어 있는 경우
    * 처리비용이 낮은 경우
    * 트랜잭션 nonce 앞 번호가 처리 되지 않았는데 뒤 번호가 대기하고 있는 경우, 뒤 트랜잭션은 대기하게 된다.

* 마이닝이 적체되어 안 지워지는 경우
    - geth를 종료
    - datadir 아래 geth디렉토리 밑의 transactions.rlp를 삭제
    - geth 다시 시작한다.


In [3]:
!geth --exec txpool.inspect attach http://117.16.44.45:8445

{
  pending: {},
  queued: {}
}


## 5.7 gas

트랜잭션을 처리하기 위해서는 비용이 발생한다. 노드들의 컴퓨팅 자원이 투입되기 때문에 이에 대한 비용이다.
비용은 송금 금액에 비례하는 것이 아니라, 데이터 크기와 소요되는 컴퓨팅 자원량에 따라 산정이 된다.
비용은 소모되는 **gas**에 **gasPrice** 단가를 곱하여 계산한다.
주유소에 비유해서 이해할 수 있다.
주행거리에 따라 필요한 연료량과 단가에 따라 주유를 하는 방식이다. 
이러한 거래에 비용이 발생하게 되면, DDos 공격 역시 상당한 비용이 수반되게 되므로 쉽게 시도하지 못하게 되는 효과가 있다.

```python
거래 비용 = gas price * gas
```

### gasPrice

gasPrice는 단가로서 사용자가 정할 수 있다. 
보통 명시하지 않으면 최선의 가격으로 정해지게 된다.
gasPrice에 따라 처리속도가 결정된다.
gasPrice가 너무 적으면 마이닝되지 않을 수 있으며, 반대로 많으면 빠르게 마이닝 될 수 있다.
gasPrice가 높을수록 더 많은 마이너가 처리하려고 들 것이고 트랜잭션이 마이닝되는 시간이 짧아지게 된다.
예를 들어, 10 Gwei로 정하면 2분 이내로 빠르게 처리된다. https://ethgasstation.info/

Speed | Gas Price (gwei)
-----|-----
SafeLow (<30m) | 4.1
Standard (<5m) | 5.6
Fast (<2m) | 10

메인네트워크에서 gasPrice 단가는 시간에 따라 정해져 있지 않고 변동한다 (참조 https://etherscan.io/chart/gasprice)

### gas limit

gas limit은 사용 한도를 말한다. 사용자가 최대로 지불할 용의가 있는 한도를 말한다.
단순한 거래의 경우 보통 30,000 정도면 충분하다.
gas가 소진될 때 까지 마이닝하고, 남으면 반환, 모자라면 **Out of Gas Exception**이 발생한다.

In [4]:
!geth --exec "eth.getBlock('latest').gasLimit" attach http://117.16.44.45:8445

8000000


### gas 산정

**실행비용 execution costs**는 사용되는 명령어를 기준으로 산정이 된다.

코드 | 명령어 | 비용 | 설명
-----|-----|-----|-----
0x00 | stop | 실행중지
0 01 | ADD | 3 | 더하기
10 | LT | 3 | less than 비교
20 | SHA3 | Keccak-256 hash 계산
30 | ADDRESS | 현재 실행 계정 주소 읽기
40 | BLOCKHASH | 최근 블록 해시 값 읽기


![alt text](figures/5_gasCost.png "gas cost in yellow paper")

**거래 비용 transaction costs**은 byte수에 따라 산정된다.
데이터 값이 없는 $G_{txdatazero}$는 4, 데이터 값이 있는 $G_{txdatanonzero}$는 68로 계산된다.
1594 * 68 + 184 * 4 = 109128로 계산한다.


개스비용을 적지 않으면, ```web3.eth.estimateGas()``` 함수로 산정한 값으로 적용하게 된다.

```python
var gas = web3.eth.estimateGas({
    data: byteCode
});
console.log(gas);
```

In [1]:
txdata = "606060405260405160808061067283398101604090815281516020830151918301516060909301519092905b42811161003457fe5b60008054600181016100468382610100565b916000526020600020900160005b8154600160a060020a038089166101009390930a928302920219161790555060008054600181016100858382610100565b916000526020600020900160005b8154600160a060020a038088166101009390930a928302920219161790555060008054600181016100c48382610100565b916000526020600020900160005b8154600160a060020a038087166101009390930a928302920219161790555060028190555b5050505061014b565b8154818355818115116101245760008381526020902061012491810190830161012a565b5b505050565b61014891905b808211156101445760008155600101610130565b5090565b90565b6105188061015a6000396000f300606060405236156100a15763ffffffff7c0100000000000000000000000000000000000000000000000000000000600035041663120b205381146100a35780634ecb35c4146100c557806366d003ac146100f557806367e404ce146101215780636cc6cde11461014d5780637022b58e1461017957806393c0e83a1461018b578063a79a3cee146101ba578063ac4c25b2146101de578063c7fe23a3146101f0575bfe5b34156100ab57fe5b6100b3610220565b60408051918252519081900360200190f35b34156100cd57fe5b6100e1600160a060020a0360043516610226565b604080519115158252519081900360200190f35b34156100fd57fe5b61010561023b565b60408051600160a060020a039092168252519081900360200190f35b341561012957fe5b610105610273565b60408051600160a060020a039092168252519081900360200190f35b341561015557fe5b6101056102ab565b60408051600160a060020a039092168252519081900360200190f35b341561018157fe5b6101896102e3565b005b341561019357fe5b610105600435610364565b60408051600160a060020a039092168252519081900360200190f35b34156101c257fe5b6100e1610396565b604080519115158252519081900360200190f35b34156101e657fe5b610189610410565b005b34156101f857fe5b6100e1600160a060020a0360043516610479565b604080519115158252519081900360200190f35b60025481565b60016020526000908152604090205460ff1681565b60006000600181548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b60006000600081548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b60006000600281548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b6102ec33610479565b15156102f85760006000fd5b600160a060020a0333166000908152600160208190526040909120805460ff19169091179055610326610396565b1561035f5761033361023b565b604051600160a060020a039182169130163190600081818185876187965a03f192505050151561035f57fe5b5b5b5b565b600080548290811061037257fe5b906000526020600020900160005b915054906101000a9004600160a060020a031681565b600080805b60005481101561040357600160006000838154811015156103b857fe5b906000526020600020900160005b9054600160a060020a036101009290920a900416815260208101919091526040016000205460ff16156103fa576001909101905b5b60010161039b565b600282101592505b505090565b33600160a060020a0316610422610273565b600160a060020a0316146104365760006000fd5b600254421161044157fe5b610333610273565b604051600160a060020a039182169130163190600081818185876187965a03f192505050151561035f57fe5b5b5b565b6000805b6000548110156104e15782600160a060020a03166000828154811015156104a057fe5b906000526020600020900160005b9054906101000a9004600160a060020a0316600160a060020a031614156104d857600191506104e6565b5b60010161047d565b600091505b509190505600a165627a7a72305820b02ae8d25b238d57e8363bfc5f30351753ca38bde86d49729841a827fb4e9d660029000000000000000000000000a4be607be0a6cceff581ef35c0cdc47a7532ff540000000000000000000000007c6b393dbb1152691b02a61378bc731662cdd9f1000000000000000000000000748ed23daa18226d872b5bd4d48ff2594fd6901c00000000000000000000000000000000000000000000000000000000597fc480"

def count_zero_bytes(data):
  count = 0
  for i in xrange(0, len(data), 2):
    byte = data[i:i+2]
    if byte == "00":
      count += 1
  return count

def count_non_zero_bytes(data):
  return (len(data) / 2) - count_zero_bytes(data)

print("zero-bytes: {0}".format(count_zero_bytes(txdata)))
print("non-zero-bytes: {0}".format(count_non_zero_bytes(txdata)))

zero-bytes: 184
non-zero-bytes: 1594


In [3]:
!geth --exec 'eth.getTransactionReceipt("0xd87121b8b0f84f7fa038cd7c1928ca6a222d14228125c90edc2493fdef4fb90b")' attach http://117.16.44.45:8445

{
  blockHash: "0xd2c51ae5dea10e50c915e9d7ccc6c117c2d14d0f38da936b62a1c38fd0494d26",
  blockNumber: 55169,
  contractAddress: null,
  cumulativeGasUsed: 21000,
  from: "0x2e49e21e708b7d83746ec676a4afda47f1a0d693",
  gasUsed: 21000,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0x1817a1775db945025d4a67a0bfcb633b5fed6a1fa76804d152db410c9140237d",
  to: "0xe36104ad419c719e356e86f94b5a7ca47a83f9e7",
  transactionHash: "0xd87121b8b0f84f7fa038cd7c1928ca6a222d14228125c90edc2493fdef4fb90b",
  tran

* 계정의 getTransactionCount
```python
getTransactionCount(fromAddress, 'pending')
```

In [4]:
!geth --exec 'eth.getTransactionCount("0x2e49e21e708b7d83746ec676a4afda47f1a0d693")' attach http://117.16.44.45:8445 

25


* 발생된 거래를 살펴보기
    * eth.getBlock(55169);
    * eth.filter('latest')

In [1]:
!geth --exec 'eth.getBlock(55169);' attach http://117.16.44.45:8445 

{
  difficulty: 6314813,
  extraData: "0xd783010504846765746887676f312e372e33856c696e7578",
  gasLimit: 4712388,
  gasUsed: 21000,
  hash: "0xd2c51ae5dea10e50c915e9d7ccc6c117c2d14d0f38da936b62a1c38fd0494d26",
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  miner: "0x2e49e21e708b7d83746ec676a4afda47f1a0d693",
  mixHash: "0x4da21d2c5696ee4d798f6445db0287a73fb2332f089ac43690d302827f52401a",
  nonce: "0x3cd421957745a4b1",
  number: 55169,
  parentHash: "0x6533c71dd0f03108ea2c657896e73253f4f60ed3c4524a2bf0009cdec35

In [2]:
!geth --exec 'eth.filter("latest");' attach http://117.16.44.45:8445 

{
  callbacks: [],
  filterId: "0xa6d9df622f0acc03602e7b3c185ea906",
  getLogsCallbacks: [],
  implementation: {
    getLogs: function(),
    newFilter: function(),
    poll: function(),
    uninstallFilter: function()
  },
  options: "latest",
  pollFilters: [],
  requestManager: {
    polls: {},
    provider: {
      newAccount: function(),
      send: function github.com/ethereum/go-ethereum/console.(*bridge).Send-fm(),
      sendAsync: function github.com/ethereum/go-ethereum/console.(*bridge).Send-fm(),
      sign: function(),
      unlockAccount: function()
    },
    timeout: null,
    poll: function(),
    reset: function(keepIsSyncing),
    send: function(data),
    sendAsync: function(data, callback),
    sendBatch: function(data, callback),
    setProvider: function(p),
    startPolling: function(data, pollId, callback, uninstall),
    stopPolling: function(pollId)
  },
  formatter: function(log),
  get: function(callback),
  stopWatching: function(),
  watch: function(callb

## 문제 5-1: 다른 계정으로 송금 거래

<a id='5-1'></a>

* 1-1 같은 기계, 다른 계정으로 송금하기
* 1-2 다른 기계, 다른 계정으로 송금하기


### 1-1 같은 기계, 다른 계정으로 송금하기

* bash shell에서 송금할 수 있다.
* 마이닝을 해서 거래를 처리하도록 한다.
* 송금하려면, account unlock해야 한다. geth를 시작할 때 --unlock 하거나, console에서 직접 할 수 있다.
```python
> personal.unlockAccount(eth.accounts[2])
```


In [1]:
!geth --exec "eth.getBalance(eth.accounts[1])" attach ethhttp://117.16.44.45:8445

185000000000000000000


In [2]:
!geth --exec "eth.sendTransaction({from:eth.accounts[1], to:eth.accounts[0],value:10000})" attach http://117.16.44.45:8445 

Error: authentication needed: password or unlock
    at web3.js:3143:20
    at web3.js:6347:15
    at web3.js:5081:36
    at <anonymous>:1:1



* 스크립트로 실행한다. 거래가 발생하고, 마이닝을 하고 나면, bloack number가 하나 증가한다.

In [4]:
%%writefile src/e_testTran.js
console.log('wei balance: ', web3.fromWei(eth.getBalance(eth.accounts[0]),"ether"));
console.log('block number: ', eth.blockNumber);
var t=eth.sendTransaction({from:eth.accounts[0], to:eth.accounts[1],value:10000});
console.log('transactionHash: ',t);
console.log('...mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done...');
console.log('block number: ', eth.blockNumber);
console.log('wei balance: ', web3.fromWei(eth.getBalance(eth.accounts[0]),"ether"));

Overwriting src/e_testTran.js


In [5]:
!geth --exec 'loadScript("src/e_testTran.js")' attach http://117.16.44.45:8445

wei balance:  337759.843749999999969
block number:  70294
transactionHash:  0x5e5ea086be169969e36f17ffe6f9fd942d255b36fa04d26c4fce815f7c7564fb
...mining start
mining done...
block number:  70295
wei balance:  337764.843749999999949
true


### 1-2 다른 기계, 다른 계정으로 송금하기

* 117.16.44.45에서 117.16.44.46 ("0x519775cc61e4c9b3f19b75426a7a3696a3c85035") 으로 송금
* 117.16.44.45에서 멀티노드의 117.16.44.46 계정 잔고를 알 수 있다.

In [7]:
!geth --exec 'eth.getBalance("0x519775cc61e4c9b3f19b75426a7a3696a3c85035");' attach http://117.16.44.45:8445 

10000


In [10]:
%%writefile src/e_testTran2.js
var hqaccount="0x519775cc61e4c9b3f19b75426a7a3696a3c85035";
console.log('hq account balance: ', eth.getBalance(hqaccount));
console.log('block number: ', eth.blockNumber);
var t=eth.sendTransaction({from:eth.accounts[0], to:hqaccount, value:10000});
console.log('transactionHash: ',t);
console.log('...mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done...');
console.log('block number: ', eth.blockNumber);
console.log('hq account balance: ', eth.getBalance(hqaccount));

Overwriting src/e_testTran2.js


In [11]:
!geth --exec 'loadScript("src/e_testTran2.js")' attach http://117.16.44.45:8445

hq account balance:  20000
block number:  70296
transactionHash:  0x15b17f04f20322cc8ee14aef9a9b617ec77935f29ce313a78ad1388241607e6c
...mining start
mining done...
block number:  70297
hq account balance:  30000
true
